In [8]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.dates as mdates
from fredapi import Fred
import dash_bootstrap_components as dbc
#dash.register_page(__name__, path='/yield_curves')

import dash_table
import dash
from dash import dcc, html, callback
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.dates as mdates
import dash_bootstrap_components as dbc
import io
import requests

# The API URL for Norwegian Yield Curve
nor_url = "https://data.norges-bank.no/api/data/GOVT_GENERIC_RATES/B.7Y+6M+5Y+3Y+3M+12M+10Y.GBON+TBIL.?format=csv&startPeriod=2000-10-17&endPeriod=2024-10-17&locale=en"

# Send a GET request to the API
nor_response = requests.get(nor_url)

# Check if the request was successful
if nor_response.status_code == 200:
    # Load the CSV data into a pandas DataFrame
    nor_csv_data = nor_response.content.decode('utf-8')
    nor_data = pd.read_csv(io.StringIO(nor_csv_data), sep=';')

    # Select and rename relevant columns
    nor_data = nor_data[['Tenor', 'TIME_PERIOD', 'OBS_VALUE']].rename(columns={'TIME_PERIOD': 'Date', 'OBS_VALUE': 'Yield'})

# Pivot and reformat Norwegian yields data
nor_yield_df = nor_data.pivot(index='Date', columns='Tenor', values='Yield').reset_index()
nor_yield_df = nor_yield_df[['Date', '3 months', '6 months', '12 months', '3 years', '5 years', '7 years', '10 years']]
nor_yield_df.rename(columns={
    '3 months': '3M', '6 months': '6M', '12 months': '1Y', '3 years': '3Y',
    '5 years': '5Y', '7 years': '7Y', '10 years': '10Y'
}, inplace=True)

# Clean up date formatting and set index
nor_yield_df['Date'] = pd.to_datetime(nor_yield_df['Date'])
nor_yield_df.set_index('Date', inplace=True)
nor_yield_df.dropna(how='all', inplace=True)
nor_yield_df.fillna(method='ffill', inplace=True)

# Resample to monthly frequency
nor_yield_monthly = nor_yield_df.resample('ME').last()


# Update with today's data
nor_today = pd.Timestamp(datetime.today().date())
if not nor_yield_monthly.index.empty and nor_yield_monthly.index[-1] > nor_today:
    nor_last_yields = nor_yield_monthly.iloc[-1]
    nor_yield_monthly.loc[nor_today] = nor_last_yields
nor_yield_monthly.sort_index(inplace=True)


# Create Dash DataTable for historical yields
nor_yield_monthly_reset = nor_yield_monthly.reset_index()
nor_yield_monthly_reset.sort_values('Date', ascending=False, inplace=True)
nor_yield_monthly_reset['Date'] = nor_yield_monthly_reset['Date'].dt.strftime('%d-%m-%Y')


C:\Users\jonas\AppData\Local\Temp\ipykernel_15792\3149188169.py:53: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  nor_yield_df.fillna(method='ffill', inplace=True)


In [9]:
nor_yield_monthly_reset


Tenor,Date,3M,6M,1Y,3Y,5Y,7Y,10Y
69,31-10-2024,4.423,4.432,4.153,3.487,3.458,3.483,3.539
68,30-09-2024,4.435,4.382,4.093,3.270,3.252,3.239,3.315
67,31-08-2024,4.407,4.318,4.194,3.217,3.162,3.222,3.312
66,31-07-2024,4.419,4.414,4.321,3.462,3.384,3.398,3.443
65,30-06-2024,4.424,4.445,4.422,3.708,3.613,3.617,3.593
...,...,...,...,...,...,...,...,...
4,31-05-2019,1.151,1.191,1.214,1.301,1.297,1.380,1.559
3,30-04-2019,1.104,1.152,1.231,1.342,1.456,1.585,1.738
2,31-03-2019,1.086,1.127,1.192,1.255,1.334,1.438,1.601
1,28-02-2019,1.027,1.039,1.065,1.143,1.375,1.563,1.718


In [12]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.dates as mdates
from fredapi import Fred
import dash_bootstrap_components as dbc

import dash_table
import dash
from dash import dcc, html, callback
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.dates as mdates
import dash_bootstrap_components as dbc
import io
import requests

# The API URL for Norwegian Yield Curve
nor_url = "https://data.norges-bank.no/api/data/GOVT_GENERIC_RATES/B.7Y+6M+5Y+3Y+3M+12M+10Y.GBON+TBIL.?format=csv&startPeriod=2000-10-17&endPeriod=2024-10-17&locale=en"

# Send a GET request to the API
nor_response = requests.get(nor_url)

# Check if the request was successful
if nor_response.status_code == 200:
    # Load the CSV data into a pandas DataFrame
    nor_csv_data = nor_response.content.decode('utf-8')
    nor_data = pd.read_csv(io.StringIO(nor_csv_data), sep=';')

    # Select and rename relevant columns
    nor_data = nor_data[['Tenor', 'TIME_PERIOD', 'OBS_VALUE']].rename(columns={'TIME_PERIOD': 'Date', 'OBS_VALUE': 'Yield'})

# Pivot and reformat Norwegian yields data
nor_yield_df = nor_data.pivot(index='Date', columns='Tenor', values='Yield').reset_index()
nor_yield_df = nor_yield_df[['Date', '3 months', '6 months', '12 months', '3 years', '5 years', '7 years', '10 years']]
nor_yield_df.rename(columns={
    '3 months': '3M', '6 months': '6M', '12 months': '1Y', '3 years': '3Y',
    '5 years': '5Y', '7 years': '7Y', '10 years': '10Y'
}, inplace=True)

# Clean up date formatting and set index
nor_yield_df['Date'] = pd.to_datetime(nor_yield_df['Date'])
nor_yield_df.set_index('Date', inplace=True)
nor_yield_df.dropna(how='all', inplace=True)
nor_yield_df.fillna(method='ffill', inplace=True)

# Resample to monthly frequency
nor_yield_monthly = nor_yield_df.resample('ME').last()


# Update with today's data
nor_today = pd.Timestamp(datetime.today().date())
if not nor_yield_monthly.index.empty and nor_yield_monthly.index[-1] > nor_today:
    nor_last_yields = nor_yield_monthly.iloc[-1]
    nor_yield_monthly.loc[nor_today] = nor_last_yields
nor_yield_monthly.sort_index(inplace=True)


# Create Dash DataTable for historical yields
nor_yield_monthly_reset = nor_yield_monthly.reset_index()
nor_yield_monthly_reset.sort_values('Date', ascending=False, inplace=True)
nor_yield_monthly_reset['Date'] = nor_yield_monthly_reset['Date'].dt.strftime('%d-%m-%Y')

nor_yield_monthly


C:\Users\jonas\AppData\Local\Temp\ipykernel_15792\3281134092.py:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  nor_yield_df.fillna(method='ffill', inplace=True)


Tenor,3M,6M,1Y,3Y,5Y,7Y,10Y
Date,,,,,,,
2019-01-31,0.985,0.985,1.045,1.170,1.372,1.555,1.707
2019-02-28,1.027,1.039,1.065,1.143,1.375,1.563,1.718
2019-03-31,1.086,1.127,1.192,1.255,1.334,1.438,1.601
2019-04-30,1.104,1.152,1.231,1.342,1.456,1.585,1.738
2019-05-31,1.151,1.191,1.214,1.301,1.297,1.380,1.559
...,...,...,...,...,...,...,...
2024-06-30,4.424,4.445,4.422,3.708,3.613,3.617,3.593
2024-07-31,4.419,4.414,4.321,3.462,3.384,3.398,3.443
2024-08-31,4.407,4.318,4.194,3.217,3.162,3.222,3.312


In [20]:
# Initialize FRED API
FRED_API_KEY = '6188d31bebbdca093493a1077d095284'
fred = Fred(FRED_API_KEY)

# Define the observation period (e.g., last 10 years)
observation_start = (datetime.today() - timedelta(days=10 * 365)).strftime('%Y-%m-%d')  # 10 years ago

# List of maturities and their corresponding FRED series IDs
maturity_labels = ['1M', '3M', '6M', '1Y', '2Y', '3Y', '5Y', '7Y', '10Y', '20Y', '30Y']
series_ids = ['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3', 'DGS5',
              'DGS7', 'DGS10', 'DGS20', 'DGS30']

# Fetch data for each series over the entire historical period
yield_data = {}
for series_id in series_ids:
    yield_data[series_id] = fred.get_series(series_id, observation_start=observation_start)

# Combine all series into a single DataFrame
yield_df = pd.DataFrame(yield_data)
yield_df.index = pd.to_datetime(yield_df.index)

# Drop rows with all NaN values and fill forward to ensure continuous data
yield_df.dropna(how='all', inplace=True)
yield_df.fillna(method='ffill', inplace=True)

# Resample data to quarterly frequency and include the last observation
yield_df_quarterly = yield_df.resample('QE').last()

# Update the last date's yield values to today's date
today = pd.Timestamp(datetime.today().date())

# If today's date already exists in the DataFrame, keep the values intact
yield_df_quarterly.sort_index(inplace=True)

# Initialize static yield table data
last_yields = yield_df_quarterly.iloc[-1]  # Get the latest yields

C:\Users\jonas\AppData\Local\Temp\ipykernel_15792\2160725705.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  yield_df.fillna(method='ffill', inplace=True)


In [21]:
import pandas as pd
from IPython.display import HTML

# Example DataFrame
data = {
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [24, 27, 22],
    'City': ['New York', 'San Francisco', 'Los Angeles']
}
df = pd.DataFrame(data)

# Convert DataFrame to HTML with a scrollable <div>
html_table = df.to_html(index=False)
scrollable_html = f"""
<div style="max-height:300px; overflow:auto; border:1px solid #ddd;">
{html_table}
</div>
"""

# Display the HTML table in the cell output
display(HTML(scrollable_html))


Name,Age,City
Alice,24,New York
Bob,27,San Francisco
Charlie,22,Los Angeles


In [19]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Define the observation period (e.g., last 10 years)
observation_start = (datetime.today() - timedelta(days=20 * 365)).strftime('%Y-%m-%d')  # 10 years ago

# List of maturities and their corresponding Yahoo Finance symbols
maturity_labels = ['3M', '2Y', '5Y', '10Y','30Y']
yahoo_symbols = {
    '3M': '^IRX',     # 3 month Treasury Bill
    '2Y': '^2YY=F',   # 2 year Treasury Note (alternative ticker)
    '5Y': '^FVX',    # 5 year Treasury Note
    '10Y': '^TNX',    # 10 year Treasury Note
    '30Y': '^TYX'     # 30 year Treasury Bond
}

# Fetch data for each series over the entire historical period
yield_data = {}
for label, symbol in yahoo_symbols.items():
    try:
        data = yf.download(symbol, start=observation_start, progress=False)
        yield_data[label] = data['Close']  # Store only the 'Close' price as yield data
    except Exception as e:
        print(f"Failed to fetch {label} ({symbol}): {e}")

# Combine all series into a single DataFrame
yield_df = pd.DataFrame(yield_data)

# Resample data to quarterly frequency and include the last observation
yield_df.index = pd.to_datetime(yield_df.index)
yield_df_quarterly = yield_df.resample('QE').last()

# Drop rows with all NaN values and fill forward to ensure continuous data
yield_df_quarterly.dropna(how='all', inplace=True)
yield_df_quarterly.fillna(method='ffill', inplace=True)

# Update the last date's yield values to today's date
today = pd.Timestamp(datetime.today().date())

# If today's date already exists in the DataFrame, keep the values intact
yield_df_quarterly.sort_index(inplace=True)

# Initialize static yield table data
last_yields = yield_df_quarterly.iloc[-1]  # Get the latest yields

# Display the most recent yield data
print(last_yields)



1 Failed download:
['^2YY=F']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


ValueError: If using all scalar values, you must pass an index

In [18]:
yield_data

{'3M': Ticker                      ^IRX
 Date                            
 2004-11-15 00:00:00+00:00  2.057
 2004-11-16 00:00:00+00:00  2.091
 2004-11-17 00:00:00+00:00  2.080
 2004-11-18 00:00:00+00:00  2.087
 2004-11-19 00:00:00+00:00  2.096
 ...                          ...
 2024-11-04 00:00:00+00:00  4.418
 2024-11-05 00:00:00+00:00  4.423
 2024-11-06 00:00:00+00:00  4.430
 2024-11-07 00:00:00+00:00  4.420
 2024-11-08 00:00:00+00:00  4.423
 
 [5026 rows x 1 columns],
 '1Y': Ticker                      ^FVX
 Date                            
 2004-11-15 00:00:00+00:00  3.531
 2004-11-16 00:00:00+00:00  3.557
 2004-11-17 00:00:00+00:00  3.481
 2004-11-18 00:00:00+00:00  3.476
 2004-11-19 00:00:00+00:00  3.564
 ...                          ...
 2024-11-04 00:00:00+00:00  4.167
 2024-11-05 00:00:00+00:00  4.172
 2024-11-06 00:00:00+00:00  4.272
 2024-11-07 00:00:00+00:00  4.192
 2024-11-08 00:00:00+00:00  4.192
 
 [5026 rows x 1 columns],
 '2Y': Empty DataFrame
 Columns: [^USTB2Y]
 Inde